# Self-feedback

In [2]:
import re
import json
from pydantic import BaseModel, Field, ValidationError
from langchain_openai import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import JsonOutputParser
from dotenv import load_dotenv, find_dotenv

load_dotenv(find_dotenv(usecwd=True))

True

In [10]:
model = ChatOpenAI(model="gpt-4o-mini")

In [4]:
# pydantic model
class FeedbackResponse(BaseModel):
    rating: str = Field(..., description="Scoring in percentage")
    feedback: str = Field(..., description="Detailed feedback")
    revised_output: str = Field(..., description="An improved output describing the key events and significance of the American Civil War")

In [16]:
# self-feedabck function
def self_feedback(user_prompt: str, max_iterations: int = 5, target_rating: int = 90):
    content = ""
    feedback = ""
    for i in range(max_iterations):
        prompt_content = user_prompt if i == 0 else ""
        messages = [
        ("system", """
    Evaluate the input in terms of how well it addresses the original task of 
    explaining the key events and significance of the American Civil War. Consider 
    factors such as: Breadth and depth of context provided; Coverage of major events; 
    Analysis of short-term and long-term impacts/consequences. 
    If you identify any gaps or areas that need further elaboration: Return output as 
    JSON with fields: 'rating': 'scoring in percentage', 'feedback': 'detailed 
    feedback', 'revised_output': 'return an improved output describing the key events 
    and significance of the American Civil War. Avoid special characters like 
    apostrophes (') and double quotes'. 
            """),
        ("user", "<prompt_content>{prompt_content}</prompt_content> <revised_output>{revised_output}</revised_output> <feedback>{feedback}</feedback>")
        ]
        prompt_template = ChatPromptTemplate.from_messages(messages)
        chain = prompt_template | model | JsonOutputParser(pydantic_object=FeedbackResponse)
        response = chain.invoke({"prompt_content": prompt_content, "revised_output": content, "feedback": feedback})

        try:
            # extract rating
            rating_num = int(re.findall(r'\d+', response['rating'])[0])

            # extract feedback and revised output
            feedback = response['feedback']
            content  = response["revised_output"]

            print(f"i={i}, Prompt Content: {prompt_content}, Rating: {rating_num} \nFeedback: {feedback}, \nRevised Output: {content}")

            if rating_num >= target_rating:
                return content
        except ValidationError as e:
            print("valid Error:", e.json())
            return "invalid response format"
    return content


In [17]:
user_prompt = "The American Civil War was a civil was in the United States between the north and the south."
result = self_feedback(user_prompt=user_prompt, max_iterations=3, target_rating=95)
result

i=0, Prompt Content: The American Civil War was a civil was in the United States between the north and the south., Rating: 10 
Feedback: The input provided is very minimal and lacks depth, context, and coverage of major events significant to the American Civil War. It fails to explain the causes, key battles, the roles of important figures, and the implications of the war on American society. There is no analysis of the short-term and long-term impacts or the significance of the war in shaping the United States. Overall, it does not address the original task adequately., 
Revised Output: The American Civil War, fought from 1861 to 1865, was a pivotal conflict in United States history between the northern states (the Union) and the southern states (the Confederacy) primarily over the issues of slavery and states rights. Key events included the attack on Fort Sumter in April 1861, which marked the beginning of the war, the Battle of Gettysburg in July 1863, one of the war's bloodiest bat

"The American Civil War took place from 1861 to 1865 and was a pivotal conflict between the northern states, known as the Union, and the southern states, known as the Confederacy. The war was fueled by deep-rooted issues such as slavery, states rights, and economic disparities. It began with the attack on Fort Sumter in April 1861, which marked the start of armed hostilities. The First Battle of Bull Run in July 1861 demonstrated the war would not be a quick affair. Key battles included the Battle of Antietam in September 1862, recognized as the bloodiest single day in American military history. This battle allowed President Abraham Lincoln to issue the Emancipation Proclamation in January 1863, which shifted the war's focus to ending slavery. The Battle of Gettysburg in July 1863 was a significant turning point as the Confederacy suffered a major defeat. Notable leaders included Generals Ulysses S Grant and Robert E Lee, who led their armies into key confrontations. The conflict concl